In [45]:
import cv2
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib qt

plt.rcParams['figure.figsize'] = [15, 15]

image = cv2.imread('./images/jeep2.jpeg')

# cell size in pixels (width,height), must be smaller than the detection window of the image
cell_size = (8,8)

#  block size must be as well < detection window
num_cells_per_bloc = (2,2)

# number of histogram bins for orientation
num_bins = 9

original_image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

block_size = (num_cells_per_bloc[0] * cell_size[0],
              num_cells_per_bloc[1] * cell_size[1])

x_cells = gray_image.shape[1] // cell_size[0]
y_cells = gray_image.shape[0] // cell_size[1]

# plt.imshow(image)

# horizontal distance between blocks in units of cell size, must be int
h_stride = 1

v_stride = 1

block_stride = (cell_size[0] * h_stride, cell_size[1] * v_stride)

# must be int and may not always be equat to image
win_size = (x_cells * cell_size[0], y_cells * cell_size[1])

# Print the shape of the gray scale image for reference
print('\nThe gray scale image has shape: ', gray_image.shape)
print()

# Print the parameters of our HOG descriptor
print('HOG Descriptor Parameters:\n')
print('Window Size:', win_size)
print('Cell Size:', cell_size)
print('Block Size:', block_size)
print('Block Stride:', block_stride)
print('Number of Bins:', num_bins)
print()

hog = cv2.HOGDescriptor(win_size,block_size,block_stride,cell_size,num_bins)

hog_descriptor = hog.compute(gray_image)

# total no. of blocks along the width of the window
tot_bx = np.uint32((x_cells - num_cells_per_bloc[0]) / h_stride + 1)
#  and height
tot_by = np.uint32(((y_cells - num_cells_per_bloc[1]) / v_stride) + 1)

#  total number of feature elements
total_els = (tot_bx) * (tot_by) * num_cells_per_bloc[0] * num_cells_per_bloc[1] * num_bins

print(total_els)

#  reshape for plotting
hog_descriptor_reshaped = hog_descriptor.reshape(tot_bx,tot_by,num_cells_per_bloc[0],num_cells_per_bloc[1],num_bins).transpose((1,0,2,3,4))

#  average gradient for each cell
ave_grad = np.zeros((y_cells,x_cells,num_bins))

#  histogram/cell array
hist_counter = np.zeros((y_cells,x_cells,1))

for i in range(num_cells_per_bloc[0]):
    for j in range(num_cells_per_bloc[1]):
        ave_grad[i:tot_by +i, j:tot_bx + j] += hog_descriptor_reshaped[:,:,i,j,:]

        hist_counter[i:tot_by, j:tot_bx + j] += 1

# calculate mean
ave_grad /= hist_counter

# calculate the total number of vectors we have in all the cells
len_vecs = ave_grad.shape[0] * ave_grad.shape[1] * ave_grad.shape[2]

# equally spaced bins between 0 and 180
deg= np.linspace(0,np.pi,num_bins,endpoint=False)

# Each cell will have a histogram with num_bins. For each cell, plot each bin as a vector (with its magnitude
# equal to the height of the bin in the histogram, and its angle corresponding to the bin in the histogram).
# To do this, create rank 1 arrays that will hold the (x,y)-coordinate of all the vectors in all the cells in the
# image. Also, create the rank 1 arrays that will hold all the (U,V)-components of all the vectors in all the
# cells in the image. Create the arrays that will hold all the vector positons and components.
U = np.zeros((len_vecs))
V = np.zeros((len_vecs))
X = np.zeros((len_vecs))
Y = np.zeros((len_vecs))

counter = 0

for i in range(ave_grad.shape[0]):
    for j in range(ave_grad.shape[1]):
        for k in range(ave_grad.shape[2]):
            U[counter] = ave_grad[i,j,k] * np.cos(deg[k])
            V[counter] = ave_grad[i,j,k] * np.sin(deg[k])

            X[counter] = (cell_size[0] / 2) + (cell_size[0] * i)
            Y[counter] = (cell_size[1] / 2) + (cell_size[1] * j)

            counter = counter + 1
# Create the bins in degress to plot our histogram.
angle_axis = np.linspace(0, 180, num_bins, endpoint = False)
angle_axis += ((angle_axis[1] - angle_axis[0]) / 2)

# Create a figure with 4 subplots arranged in 2 x 2
fig, ((a,b),(c,d)) = plt.subplots(2,2)

# Set the title of each subplot
a.set(title = 'Gray Scale Image\n(Click to Zoom)')
b.set(title = 'HOG Descriptor\n(Click to Zoom)')
c.set(title = 'Zoom Window', xlim = (0, 18), ylim = (0, 18), autoscale_on = False)
d.set(title = 'Histogram of Gradients')

# Plot the gray scale image
a.imshow(gray_image, cmap = 'gray')
a.set_aspect(aspect = 1)

# Plot the feature vector (HOG Descriptor)
b.quiver(Y, X, U, V, color = 'white', headwidth = 0, headlength = 0, scale_units = 'inches', scale = 5)
b.invert_yaxis()
b.set_aspect(aspect = 1)
b.set_facecolor('white')

# Define function for interactive zoom
def onpress(event):

    #Unless the left mouse button is pressed do nothing
    if event.button != 1:
        return

    # Only accept clicks for subplots a and b
    if event.inaxes in [a, b]:

        # Get mouse click coordinates
        x, y = event.xdata, event.ydata

        # Select the cell closest to the mouse click coordinates
        cell_num_x = np.uint32(x / cell_size[0])
        cell_num_y = np.uint32(y / cell_size[1])

        # Set the edge coordinates of the rectangle patch
        edgex = x - (x % cell_size[0])
        edgey = y - (y % cell_size[1])

        # Create a rectangle patch that matches the the cell selected above
        rect = patches.Rectangle((edgex, edgey),
                                  cell_size[0], cell_size[1],
                                  linewidth = 1,
                                  edgecolor = 'magenta',
                                  facecolor='none')

        # A single patch can only be used in a single plot. Create copies
        # of the patch to use in the other subplots
        rect2 = copy.copy(rect)
        rect3 = copy.copy(rect)

        # Update all subplots
        a.clear()
        a.set(title = 'Gray Scale Image\n(Click to Zoom)')
        a.imshow(gray_image, cmap = 'gray')
        a.set_aspect(aspect = 1)
        a.add_patch(rect)

        b.clear()
        b.set(title = 'HOG Descriptor\n(Click to Zoom)')
        b.quiver(Y, X, U, V, color = 'white', headwidth = 0, headlength = 0, scale_units = 'inches', scale = 5)
        b.invert_yaxis()
        b.set_aspect(aspect = 1)
        b.set_facecolor('black')
        b.add_patch(rect2)

        c.clear()
        c.set(title = 'Zoom Window')
        c.quiver(Y, X, U, V, color = 'white', headwidth = 0, headlength = 0, scale_units = 'inches', scale = 1)
        c.set_xlim(edgex - cell_size[0], edgex + (2 * cell_size[0]))
        c.set_ylim(edgey - cell_size[1], edgey + (2 * cell_size[1]))
        c.invert_yaxis()
        c.set_aspect(aspect = 1)
        c.set_facecolor('black')
        c.add_patch(rect3)

        d.clear()
        d.set(title = 'Histogram of Gradients')
        d.grid()
        d.set_xlim(0, 180)
        d.set_xticks(angle_axis)
        d.set_xlabel('Angle')
        d.bar(angle_axis,
              ave_grad[cell_num_y, cell_num_x, :],
              180 // num_bins,
              align = 'center',
              alpha = 0.5,
              linewidth = 1.2,
              edgecolor = 'k')

        fig.canvas.draw()

# Create a connection between the figure and the mouse click
fig.canvas.mpl_connect('button_press_event', onpress)
plt.show()


The gray scale image has shape:  (424, 640)

HOG Descriptor Parameters:

Window Size: (640, 424)
Cell Size: (8, 8)
Block Size: (16, 16)
Block Stride: (8, 8)
Number of Bins: 9

147888


C:\Users\Andrei\AppData\Local\Temp\ipykernel_4612\1924311164.py:86: RuntimeWarning: divide by zero encountered in divide
  ave_grad /= hist_counter
C:\Users\Andrei\AppData\Local\Temp\ipykernel_4612\1924311164.py:110: RuntimeWarning: invalid value encountered in double_scalars
  V[counter] = ave_grad[i,j,k] * np.sin(deg[k])
